<a href="https://www.kaggle.com/code/katzurasharma/vgg-16-modified?scriptVersionId=117180259" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import tensorflow as tf
import seaborn as sns

In [ ]:

path_normal = "/kaggle/input/whowhohaaha/archive/COVID-19_Radiography_Dataset/Normal"
path_covid = "/kaggle/input/whowhohaaha/archive/COVID-19_Radiography_Dataset/COVID"
len_normal = len(os.listdir(path_normal))
len_covid = len(os.listdir(path_covid))


In [ ]:
import glob
labels = ['Normal', 'COVID']
img1 = path_normal+"/Normal-1.png"
img2 = path_covid+"/COVID-1.png"
imgs = [img1, img2]

fig, ax = plt.subplots(2, figsize=(15, 15))
ax = ax.ravel()
plt.tight_layout()

for i in range(0,2):

    ax[i].imshow(plt.imread(imgs[i]), cmap='gray')
    ax[i].set_title(labels[i])

# PIL to enhance images

In [ ]:
from PIL import Image, ImageOps
def resize_image(imgpath):
    img = np.array(ImageOps.grayscale(Image.open(imgpath)).resize((64, 64), Image.ANTIALIAS))
    return img

### Using defined function to resize images

In [ ]:
import os
x_ = list()
y = list()

for i in os.listdir(path_normal):
    try:
        imgpath = path_normal+"/"+i
        img = resize_image(imgpath)
        x_.append(img)
        y.append(0)
    except:
        None
        
for i in os.listdir(path_covid):
    try:
        imgpath = path_covid+"/"+i
        img = resize_image(imgpath)
        x_.append(img)
        y.append(1)
    except:
        None

x_ = np.array(x_)

# Split test, train, validation data

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_,y,test_size = 0.2,random_state = 20)
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size = 0.15,random_state = 40)

In [ ]:
from keras.utils.np_utils import to_categorical

y = to_categorical(y,num_classes = 2)

classNames = ["normal","COVID_19"]
print(classNames)

In [ ]:
unique, counts = np.unique(y_train.argmax(axis=1), return_counts=True)
uniqueVal, countsVal = np.unique(y_val.argmax(axis=1), return_counts=True)
dict(zip(unique, counts))

In [ ]:
x_train = x_train.reshape(x_train.shape[0], *(64, 64, 1))
x_test = x_test.reshape(x_test.shape[0], *(64, 64, 1))

In [ ]:
x_train.shape

In [ ]:
dict(zip(uniqueVal, countsVal))

In [ ]:
# model1 = Sequential()

# model1.add(Conv2D(32, (3, 3), input_shape = (64,64,1), activation='relu'))
# model1.add(MaxPooling2D(pool_size = (2, 2)))
# model1.add(Dropout(0.2))

# model1.add(Conv2D(64, (3, 3), activation='relu'))
# model1.add(MaxPooling2D(pool_size = (2, 2)))
# model1.add(Dropout(0.2))

# model1.add(Conv2D(128, (3, 3), activation='relu'))
# model1.add(MaxPooling2D(pool_size = (2, 2)))
# model1.add(Dropout(0.2))

# model1.add(Flatten())

# model1.add(Dense(128, activation = 'relu'))
# model1.add(Dense(2, activation = 'sigmoid')) 
# DEnse(2 (cuz 2 classes))

In [ ]:
# model1.compile(loss ='binary_crossentropy', optimizer='adam',metrics =['acc'])
# model1.summary()

In [ ]:
# history1 = model1.fit(x_train, y_train, batch_size = 32, epochs = 20, verbose = 1, validation_data = (x_test, y_test))

In [ ]:
# model1.save('model1.hdf5')

# Build model VGG16 # 

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout
from keras.layers import Dense, Flatten
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD

In [ ]:
modelvgg = Sequential()

modelvgg.add(Conv2D(32, (3, 3), input_shape = (64,64,1), activation='relu'))
modelvgg.add(Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(MaxPooling2D((2, 2)))

modelvgg.add(SeparableConv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(SeparableConv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(BatchNormalization())
modelvgg.add(MaxPooling2D((2, 2)))

modelvgg.add(SeparableConv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(SeparableConv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(BatchNormalization())
modelvgg.add(MaxPooling2D((2, 2)))

modelvgg.add(SeparableConv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(SeparableConv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(BatchNormalization())
modelvgg.add(MaxPooling2D((2, 2)))
modelvgg.add(Dropout(rate=0.2))

modelvgg.add(SeparableConv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(SeparableConv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(BatchNormalization())
modelvgg.add(MaxPooling2D((2, 2)))
modelvgg.add(Dropout(rate=0.2))

modelvgg.add(Flatten())
modelvgg.add(Dense(512, activation="relu"))  
modelvgg.add(Dropout(rate=0.7))
modelvgg.add(Dense(128, activation="relu"))
modelvgg.add(Dropout(0.5))
modelvgg.add(Dense(2, activation="sigmoid"))
#     model  = Model(inputs=_input, outputs=output)
modelvgg.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])








# model1.add(Conv2D(32, (3, 3), input_shape = (28,28,1), activation='relu'))
# model1.add(MaxPooling2D(pool_size = (2, 2)))
# model1.add(Dropout(0.2))

# model1.add(Conv2D(64, (3, 3), activation='relu'))
# model1.add(MaxPooling2D(pool_size = (2, 2)))
# model1.add(Dropout(0.2))

# model1.add(Conv2D(128, (3, 3), activation='relu'))
# model1.add(MaxPooling2D(pool_size = (2, 2)))
# model1.add(Dropout(0.2))

# model1.add(Flatten())

# model1.add(Dense(128, activation = 'relu'))
# model1.add(Dense(25, activation = 'softmax'))

In [ ]:
modelvgg.summary()

In [ ]:
history = modelvgg.fit(x_train, y_train, validation_data=(x_test, y_test),
                            batch_size=32, epochs=20)

In [ ]:
def showChartEpochAccuracy(history):
    # show a nicely formatted classification report
    print("[INFO] evaluating network...")
    # plot the training loss and accuracy
    N = 20
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
    plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
    plt.title("Training Loss and Accuracy Dataset")
    plt.xlabel("Epochs #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.show()

showChartEpochAccuracy(history)

In [ ]:
mypredict = modelvgg.predict(x_test)
pred = np.argmax(mypredict, axis=-1)
print(pred)

In [ ]:
evalute = modelvgg.evaluate(x_test, y_test)

In [ ]:
print("Accuracy: {:.2f}%".format(evalute[1] * 100))  
print("Loss: {}".format(evalute[0])) 

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 


cm = confusion_matrix(y_test.argmax(axis=1), mypredict.argmax(axis=1))

plt.figure(figsize=(24,12))

plt.suptitle("Confusion Matrix",fontsize=24)
plt.title("Covid Detection with VGG-16 Confusion Matrix")
sns.heatmap(cm,annot=True,cmap="Blues",fmt="d",cbar=False, annot_kws={"size": 24})

plt.show()


In [ ]:
print ('Accuracy Score :',accuracy_score(y_test.argmax(axis=1), mypredict.argmax(axis=1)) )
print ('Report : ')
print (classification_report(y_test.argmax(axis=1), mypredict.argmax(axis=1)) )

In [ ]:
modelvgg.save("modelvgg2.h5")